<a href="https://colab.research.google.com/github/johnjustine5646/SentimentAnalysis_Lab7/blob/main/John_522_Lab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) Find the similarity between two documents

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

with open('/content/text1.txt', 'r', encoding='utf-8') as file:
    doc1 = file.read()

with open('/content/text2.txt', 'r', encoding='utf-8') as file:
    doc2 = file.read()

documents = [doc1, doc2]

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

cos_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix)
print("Cosine Similarity between the two documents:", cos_sim[0, 1])

Cosine Similarity between the two documents: 0.4356132262843411


In [2]:
def jaccard_similarity(doc1, doc2):
    set1 = set(doc1.split())
    set2 = set(doc2.split())
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union

jac_sim = jaccard_similarity(doc1, doc2)
print("Jaccard Similarity between the two documents:", jac_sim)

Jaccard Similarity between the two documents: 0.09649122807017543


In [3]:
import nltk
nltk.download('stopwords')
import nltk
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import defaultdict

df = pd.read_csv('/content/Tweets.csv')

dataset = df[['text', 'airline_sentiment']]

stop_words = set(stopwords.words('english'))
word_freq = defaultdict(lambda: [0, 0])
for _, row in dataset.iterrows():
    text = row['text']
    label = row['airline_sentiment']
    words = [word.lower() for word in word_tokenize(text) if word.isalnum() and word.lower() not in stop_words]
    for word in words:
        word_freq[word][label == 'positive'] += 1

total_positive = sum(word_freq[word][1] for word in word_freq)
total_negative = sum(word_freq[word][0] for word in word_freq)
prior_positive = total_positive / (total_positive + total_negative)
prior_negative = total_negative / (total_positive + total_negative)

def classify(text):
    words = [word.lower() for word in word_tokenize(text) if word.isalnum() and word.lower() not in stop_words]
    log_prob_positive = sum([word_freq[word][1] / total_positive for word in words])
    log_prob_negative = sum([word_freq[word][0] / total_negative for word in words])
    prob_positive = prior_positive * log_prob_positive
    prob_negative = prior_negative * log_prob_negative
    return 'positive' if prob_positive > prob_negative else 'negative'

test_data = ["@VirginAmerica seriously would pay $30 a flight for seats that didn't have this playing. it's really the only bad thing about flying VA"]
for text in test_data:
    sentiment = classify(text)
    print(f"Sentiment of '{text}': {sentiment}")


Sentiment of '@VirginAmerica seriously would pay $30 a flight for seats that didn't have this playing. it's really the only bad thing about flying VA': negative


## Sentiment Analysis

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam


In [6]:
df = pd.read_csv('/content/Tweets.csv')

In [7]:
texts = df['text'].tolist()
labels = df['airline_sentiment'].tolist()

texts = df['text'].tolist()
labels = df['airline_sentiment'].tolist()

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

labels_one_hot = to_categorical(labels_encoded)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

max_len = max(len(seq) for seq in sequences)
sequences_padded = pad_sequences(sequences, maxlen=max_len, padding='post')

X_train, X_test, y_train, y_test = train_test_split(sequences_padded, labels_one_hot, test_size=0.2, random_state=42)

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=32, input_length=max_len))
model.add(LSTM(32))
model.add(Dense(3, activation='softmax'))  # Use 3 neurons in the output layer, with a softmax activation function

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)  # Adjust batch size for efficiency

loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/10
366/366 [==============================] - 11s 23ms/step - loss: 0.7493 - accuracy: 0.6841
Epoch 2/10
366/366 [==============================] - 9s 24ms/step - loss: 0.4885 - accuracy: 0.8043
Epoch 3/10
366/366 [==============================] - 7s 20ms/step - loss: 0.3465 - accuracy: 0.8762
Epoch 4/10
366/366 [==============================] - 9s 24ms/step - loss: 0.2421 - accuracy: 0.9209
Epoch 5/10
366/366 [==============================] - 8s 22ms/step - loss: 0.1810 - accuracy: 0.9430
Epoch 6/10
366/366 [==============================] - 8s 22ms/step - loss: 0.1433 - accuracy: 0.9573
Epoch 7/10
366/366 [==============================] - 8s 23ms/step - loss: 0.1188 - accuracy: 0.9641
Epoch 8/10
366/366 [==============================] - 7s 20ms/step - loss: 0.1080 - accuracy: 0.9688
Epoch 9/10
366/366 [==============================] - 8s 23ms/step - loss: 0.0935 - accuracy: 0.9729
Epoch 10/10
92/92 [==============================] - 1s 6ms/step - loss: 0.9895 - accuracy

## 4. Implement the Sentiment Analysis using LSTM.

In [8]:


import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, SpatialDropout1D
from keras.callbacks import EarlyStopping


In [9]:
data = pd.read_csv("/content/Tweets.csv")

In [10]:
X = data['text']
y = pd.get_dummies(data['airline_sentiment']).values

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, maxlen=200)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Embedding(5000, 128, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stop])

loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy:', accuracy)

Epoch 1/10
183/183 [==============================] - 139s 745ms/step - loss: 0.7267 - accuracy: 0.6937 - val_loss: 0.5526 - val_accuracy: 0.7835
Epoch 2/10
183/183 [==============================] - 139s 761ms/step - loss: 0.4775 - accuracy: 0.8150 - val_loss: 0.4984 - val_accuracy: 0.8009
Epoch 3/10
183/183 [==============================] - 150s 819ms/step - loss: 0.3764 - accuracy: 0.8571 - val_loss: 0.4926 - val_accuracy: 0.8033
Epoch 4/10
183/183 [==============================] - 140s 765ms/step - loss: 0.3245 - accuracy: 0.8784 - val_loss: 0.5051 - val_accuracy: 0.8074
Epoch 5/10
183/183 [==============================] - 134s 735ms/step - loss: 0.2798 - accuracy: 0.8956 - val_loss: 0.5392 - val_accuracy: 0.8033
Epoch 6/10
183/183 [==============================] - 138s 754ms/step - loss: 0.2433 - accuracy: 0.9101 - val_loss: 0.5859 - val_accuracy: 0.7982
Epoch 6: early stopping
Test Accuracy: 0.8032786846160889
